<a href="https://colab.research.google.com/github/taavip/PLANK/blob/main/PLANK_statistika_kvaliteedi_analyys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install owslib geopandas pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 6.0 MB/s eta 0:00:00


In [32]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

# WFS teenuse URL
url = 'https://planeeringud.ee/plank/wfs?'

# Loome ühenduse WFS teenusega
wfs = WebFeatureService(url=url, version="1.1.0")

# Loeme kõik saadaval olevad kihid teenuses
layers = list(wfs.contents)

# Kuvame kõik saadaval olevad kihid
print("Saadaval kihid WFS teenuses:")
for layer in layers:
    print(layer)

# Loome sõnastiku, kuhu salvestame iga kihi GeoDataFrame'i objekti
gdfs = {}

# Laeme kõik kihid ja salvestame GeoDataFrame'id
for layer_name in layers:
    print(f"\nLaen kihti: {layer_name}")  # Kuvame, millist kihti hetkel laeme

    # Määrame päringu parameetrid
    params = dict(service='WFS', version="1.1.0", request='GetFeature',
                  typeNames=layer_name, count=100000, startIndex=0)  # Saame kuni 10000 rida

    # Parameetriga URL koostamine
    wfs_request_url = Request('GET', url, params=params).prepare().url

    # Laeme andmed GeoPandasiga, lisades try-except ploki
    try:
        # Kasutame requests.get, et käsitleda vastuse kodeeringut
        response = gpd.read_file(wfs_request_url)

        # Parandame võimaliku kodeeringu, kui GeoPandas ei suuda õigesti määrata
        if isinstance(response, str):
            response = response.encode('utf-8').decode('utf-8')

        gdf = gpd.read_file(wfs_request_url)

    except IndexError:
        print(f"Skipping layer {layer_name} due to IndexError (likely no features found).")
        continue  # Liigume järgmise kihi juurde

    except Exception as e:
        print(f"Error loading layer {layer_name}: {e}")
        continue  # Liigume järgmise kihi juurde

    # Kuvame, millised veerud (atribuutide) on kihi andmetes
    print(f"Veerud kihi {layer_name} kohta:")
    print(gdf.columns)  # Kuvame kõik kihi atribuudid (veerud)

    # Salvestame GeoDataFrame'i sõnastikku
    gdfs[layer_name] = gdf


Saadaval kihid WFS teenuses:
plank:yrpmsp
plank:yrpmsp_kihid
plank:yrpmsp_kihid_2
plank:yrpmsp_kihid_1
plank:maakonnaplaneeringu_ala
plank:mp_msp
plank:mp_msp_2
plank:mp_msp_1
plank:mp_tuuleala
plank:mp_vesiviljelus
plank:mp_rohev
plank:mp_toimepiirkond
plank:mp_linnaline
plank:mp_suunis
plank:mp_suunis_2
plank:mp_suunis_1
plank:mp_vaartala
plank:mp_vaartala_2
plank:mp_vaartala_1
plank:mp_taristu
plank:mp_taristu_2
plank:mp_taristu_1
plank:mp_keskus
plank:yldplaneeringud
plank:yp_tiheasustus
plank:yp_maakasutus
plank:yp_dpkoko
plank:yp_ekv
plank:yp_ekv_1
plank:yp_tingimus
plank:yp_tingimus_2
plank:yp_tingimus_1
plank:yp_ehitis
plank:yp_ehitis_2
plank:yp_ehitis_1
plank:yp_transport_3
plank:yp_transport
plank:yp_transport_1
plank:detailplaneering
plank:dp_krunt
plank:dp_hoonestus
plank:dp_tingimus
plank:dp_tingimus_2
plank:dp_tingimus_1
plank:dp_transp_3
plank:dp_transp
plank:dp_transp_1
plank:dp_tehno_3
plank:dp_tehno
plank:dp_tehno_1
plank:kep_ala
plank:kep_krunt
plank:kep_hoonestus
pl

In [33]:
gdfs['plank:detailplaneering'].head()

,gml_id,lowerCorner,upperCorner,sysid,planid,kovid,plannim,korraldaja,planseis_nimi,planksh,muutev,planeesm,algatkp,vastuvkp,kehtestkp,url,failid,planviide,tyhistkp,geometry
0,detailplaneering.30103271,6604090.780000 592842.450000,6604572.430000 593241.110000,30103271,5050.0,15,Hara kÃ¼la Kadapiku kinnistu detailplaneering,Kuusalu Vallavalitsus,kehtiv,ei,ei,"katastriÃ¼ksuse kruntideks jagamine, sihtotsta...",2002-04-18,2002-10-15,2003-01-23,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((6604549.71 593241.11, 6604495.85 593..."
1,detailplaneering.30101799,6405682.170000 621397.610000,6405747.660000 621485.970000,30101799,NaN,DP2001-002,"Valga linna kvartalis nr 15 Kungala 1a, 3, 5, ...",Valga Vallavalitsus,kehtiv,ei,ei,Kruntide piiride määramine,2001-03-07,2001-06-20,2001-08-30,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((6405738.29 621451.05, 6405747.66 621..."
2,detailplaneering.30100972,6595534.940000 582521.440000,6595722.100000 582744.540000,30100972,326.0,20,Männisalu 1 ja 2,Kuusalu Vallavalitsus,kehtiv,ei,ei,jagamine elamukruntideks,2002-06-26,None,2003-01-30,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((6595654.54 582737.99, 6595608.34 582..."
3,detailplaneering.30102577,6582856.460000 556888.110000,6583082.480000 557210.770000,30102577,151.0,1/05,"Põllu ja Vilja, Kulli",Raasiku Vallavalitsus,kehtiv,ei,ei,elamukruntideks jagamine,2005-01-11,2006-08-10,2006-10-10,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((6582930.85 556907.88, 6582940.79 556..."
4,detailplaneering.30100074,6591297.990000 578582.900000,6591408.350000 578731.270000,30100074,NaN,None,Kiiu aleviku Vana-Narva mnt 11 kinnistu detail...,Kuusalu Vallavalitsus,kehtiv,ei,ei,Vana-Narva mnt 11 kinnistule ehitusõiguse sead...,2017-10-27,2022-05-19,2022-12-15,https://planeeringud.ee/#/planning/detail/3010...,https://planeeringud.ee/plank-web/v1/planeerin...,None,None,"POLYGON ((6591297.99 578632.22, 6591408.35 578..."


In [34]:
layer_name='plank:detailplaneering'